<a href="https://colab.research.google.com/github/lsofiadb/Bootstrapping_algorithm/blob/main/Algoritmo_Bootstrapping_en_la_seleccion_de_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso 1: Generación de subconjuntos de atributos

# Paso 1: Generación de subconjuntos de atributos

In [ ]:
import pandas as pd
import random

def generate_random_subsets(file_path, max_attributes_per_subset, num_subsets):
    data = pd.read_csv(file_path)
    attributes = list(data.columns)
    if max_attributes_per_subset > len(attributes):
        max_attributes_per_subset = len(attributes)
    subsets = []
    while len(subsets) < num_subsets:
        subset = []
        cont = 0
        subset.append(attributes[len(attributes)-1])
        while cont < max_attributes_per_subset-1:
            cont+=1
            attribute = random.choice(attributes)
            if attribute not in subset:
                subset.append(attribute)
        if subset not in subsets:
            subsets.append(subset)
    dataSubFrames=[]
    for sub in subsets:
      dataSubFrames.append(data[sub])

    return dataSubFrames

# **Paso 2** **Evaluación de subconjuntos**

In [ ]:
#Cambiar para recibir dataframes

import pandas as pd

def sort_attributes_by_correlation_subset(data):

    # Calcular la matriz de correlación de los datos
    correlation_matrix = data.corr()

    # Obtener las parejas de atributos y sus índices de correlación
    attribute_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            attribute_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], abs(correlation_matrix.iloc[i, j])))

    # Ordenar las parejas de atributos por su índice de correlación de mayor a menor
    attribute_pairs_sorted = sorted(attribute_pairs, key=lambda x: abs(x[2]), reverse=True)

    # Eliminar las parejas de atributos duplicadas
    attribute_pairs_unique = []
    for pair in attribute_pairs_sorted:
        if (pair[0], pair[1]) not in attribute_pairs_unique and (pair[1], pair[0]) not in attribute_pairs_unique and pair[0] != pair[1]:
            attribute_pairs_unique.append((pair[0], pair[1], pair[2]))

    # Crear un dataframe con las parejas de atributos y sus índices de correlación
    df = pd.DataFrame(attribute_pairs_unique, columns=["Atributo 1", "Atributo 2", "Índice de correlación"])

    #Sumar indice de correlación
    sumaCorr = df["Índice de correlación"].sum()

    #Dataframe ordenado por su índice de correlación
    df.sort_values(by="Índice de correlación", ascending=False).reset_index(drop=True)

    #Retornar suma
    return sumaCorr



In [ ]:
#Correlación inicial dataset original
import pandas as pd

def sort_attributes_by_correlation(data):

    # Calcular la matriz de correlación de los datos
    correlation_matrix = data.corr()

    # Obtener las parejas de atributos y sus índices de correlación
    attribute_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            attribute_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], abs(correlation_matrix.iloc[i, j])))

    # Ordenar las parejas de atributos por su índice de correlación de mayor a menor
    attribute_pairs_sorted = sorted(attribute_pairs, key=lambda x: abs(x[2]), reverse=True)

    # Eliminar las parejas de atributos duplicadas
    attribute_pairs_unique = []
    for pair in attribute_pairs_sorted:
        if (pair[0], pair[1]) not in attribute_pairs_unique and (pair[1], pair[0]) not in attribute_pairs_unique and pair[0] != pair[1]:
            attribute_pairs_unique.append((pair[0], pair[1], pair[2]))

    # Crear un dataframe con las parejas de atributos y sus índices de correlación
    df = pd.DataFrame(attribute_pairs_unique, columns=["Atributo 1", "Atributo 2", "Índice de correlación"])

    sumaCorr = df["Índice de correlación"].sum()

    df.sort_values(by="Índice de correlación", ascending=False).reset_index(drop=True)

    # Devolver el dataframe ordenado por su índice de correlación
    return sumaCorr


# Calcular bondades

In [ ]:
def calculoBondades(originalCorr, listCorrSub):
  listBondades = []
  for i in listCorrSub:
    listBondades.append((i*100)/originalCorr)
  return listBondades

# **Paso 3: Calculo de pesos**

# **PROGRAMA** **PRINCIPAL**

In [ ]:
#Creación del data Frame
dataGlobal = pd.read_csv("sample_data/carro.csv")

#Paso 1
subDataFrames=generate_random_subsets("sample_data/carro.csv", 5, 10)

#Paso 2
#Calcular suma de relación dataset original
sumCorr=sort_attributes_by_correlation(dataGlobal)

print(sumCorr)
#lista de suma de indices de correlación por subconjunto

listIndexCorr = []
attributesxsubset =[]

for i in subDataFrames:
  #print(sort_attributes_by_correlation_subset(i))
  listIndexCorr.append(sort_attributes_by_correlation_subset(i))
  attributesxsubset.append(list(i.columns))

print(listIndexCorr)

#Lista de bondades de cada subconjunto
listBondades= calculoBondades(sumCorr,listIndexCorr )

#creación de lista de tuplas de Subconjunto y bondad
listBondadesXsubsets = []
for i in range(len(attributesxsubset)):
  listBondadesXsubsets.append((attributesxsubset[i],"Bondad",listBondades[i]))

#Imprimir tupla de subconjunto con bondad
#for it in listBondadesXsubsets:
  #print(it)
  #print("\n")

#atributos del dataset original
attributes = list(dataGlobal.columns)

#print(listBondadesXsubsets[3][0])

#for item in listBondadesXsubsets:
  #for attribute in range(len(listBondadesXsubsets[item][0])):
   # print(attribute)

#Se calculan los pesos contando las veces que aparece cada atrubuto
#en cada subconjunto
listResult = []
listAtPe = []
#Inicio For
for att in attributes:
  #print(att)
  listAtPe.append(att)
  peso = 0
  contAttr=0
  for i in range(len(listBondadesXsubsets)):
        if att in listBondadesXsubsets[i][0]:
          peso+=listBondadesXsubsets[i][2]
          contAttr+=1
          #print(peso)
  listAtPe.append(peso/contAttr)
  pesoFinal=peso/contAttr
  listResult.append([att,pesoFinal])
#Fin For

print(listResult)

# Crear un dataframe con las parejas de atributos y sus índices de correlación
pesosAttDataFrame = pd.DataFrame(listResult, columns=["Atributo", "Peso"])

pesosAttDataFrame=pesosAttDataFrame.sort_values(by="Peso", ascending=False).reset_index(drop=True)

print(pesosAttDataFrame)


4.481538352761005
[1.6110689838504317, 1.7209696684696838, 1.7205100629323207, 0.8836087339379545, 1.0090302258596835, 0.3982607177428026, 0.9811611242879897, 0.9811611242879897, 0.5007745476923584, 1.3295923802777945]
[['edad', 21.15729509605511], ['salario', 29.840576152251753], ['distancia', 26.979819128252196], ['buses', 26.26319816003014], ['transbordos', 37.17516609299237], ['Field Name', 24.848917252885297]]
      Atributo       Peso
0  transbordos  37.175166
1      salario  29.840576
2    distancia  26.979819
3        buses  26.263198
4   Field Name  24.848917
5         edad  21.157295


In [ ]:
print(pesosAttDataFrame)

      Atributo       Peso
0  transbordos  37.175166
1      salario  29.840576
2    distancia  26.979819
3        buses  26.263198
4   Field Name  24.848917
5         edad  21.157295


# Codigos de pruebas

In [ ]:
import pandas as pd

def calculate_correlation_matrix(file_path):
    # Cargar los datos desde un archivo CSV
    data = pd.read_csv(file_path)

    # Calcular la matriz de correlación de los datos
    correlation_matrix = data.corr()


    # Devolver la matriz de correlación
    return correlation_matrix